In [3]:
import graphlab as gl
import numpy as np
import pandas as pd
import os
from pylab import *

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1489207839.log


This non-commercial license of GraphLab Create for academic use is assigned to victor.li.zhu@rutgers.edu and will expire on October 06, 2017.


In [ ]:
# input parameters
#=================================
classifier_list = ['LR']
#=================================

window_length_list = [100, 150, 200, 250, 300]
window_step = 50
feature_type_list = [1, 2, 3, 12, 13, 23, 123]  # 1: dg, 2: cc, 3: pl, 12: dg+cc, 13: dg+pl, 23: cc+pl, 123: dg+cc+pl
l2_penalty_list = np.logspace(-2, 1.5, num=15, endpoint=True, base=10.0, dtype=None).tolist()
l1_penalty = 0.0

for mouse_id_test in range(1,2):

    for win_ln_idx in range(len(window_length_list)): # NOTE HERE!!!!! REMOVE "1" AFTER THIS SESSION!!!!!!!!!!!!!
        window_length = window_length_list[win_ln_idx]
        
        ##========================= load features
        # print on screen: progress
        current_progress = ("\n========================================\nLoading features for testing mouse%02d, window_length%03d ...\n" \
                            % (mouse_id_test, window_length))
        print current_progress

        # load and organize feature matrix
        comb_train, comb_test = lz_load_feature_matrix_across_subject(mouse_id_test, window_length, window_step)       
        
        for classifier_idx in range(len(classifier_list)):
            classifier = classifier_list[classifier_idx]

            # initiate
            AC = [[0] * len(l2_penalty_list) for _ in range(len(feature_type_list))] 
            SE = [[0] * len(l2_penalty_list) for _ in range(len(feature_type_list))] 
            SP = [[0] * len(l2_penalty_list) for _ in range(len(feature_type_list))] 

            for feature_type_idx in range(len(feature_type_list)):
                feature_type = feature_type_list[feature_type_idx]

                ##========================= classification
                for l2_penalty_idx in range(len(l2_penalty_list)):
                    l2_penalty = l2_penalty_list[l2_penalty_idx]

                    # print on screen: progress
                    current_progress = ("\n----------------------------------------\nClassifying --> classifier: %s, feature_type%03d, l2_penalty_%f ..." \
                                        % (classifier, feature_type, l2_penalty))
                    print current_progress

                    if classifier == 'LR':
                        AC[feature_type_idx][l2_penalty_idx],SE[feature_type_idx][l2_penalty_idx],SP[feature_type_idx][l2_penalty_idx] = \
                            lz_logistic_across_subjects_AC_SE_SP(comb_train, comb_test, feature_type, l2_penalty, l1_penalty)
                    if classifier == 'SVM':
                        AC[feature_type_idx][l2_penalty_idx],SE[feature_type_idx][l2_penalty_idx],SP[feature_type_idx][l2_penalty_idx] = \
                            lz_svm_across_subjects_AC_SE_SP(comb_train, comb_test, feature_type, l2_penalty)
                    # if classifier == 'RF'
                        # ...
                    # if classifier == 'KNN'
                        # ...

            ##====================== save results
            AC_saveName = ("%s_across_subject_test_mouse%02d_window_length%03d_AC.csv" \
                           % (classifier,mouse_id_test,window_length))
            SE_saveName = ("%s_across_subject_test_mouse%02d_window_length%03d_SE.csv" \
                           % (classifier,mouse_id_test,window_length))
            SP_saveName = ("%s_across_subject_test_mouse%02d_window_length%03d_SP.csv" \
                           % (classifier,mouse_id_test,window_length))
            df_AC = pd.DataFrame(AC)
            df_SE = pd.DataFrame(SE)        
            df_SP = pd.DataFrame(SP)  
            
            os.chdir("/Users/lizhu/Dropbox/projects/calcium/classification_result/across_subject") 
            df_AC.to_csv(AC_saveName, index = False, header = False)
            df_SE.to_csv(SE_saveName, index = False, header = False)
            df_SP.to_csv(SP_saveName, index = False, header = False)


Loading features for testing mouse01, window_length100 ...



In [63]:
# load and organize feature matrix
def lz_load_feature_matrix_across_subject(mouse_id_test, window_length, window_step):
    # the mouse_id is the one that saved for the testing set
    # the mice other than the mouse_id are used for the training set
    
    mouse_id_full = range(1,7)
    mouse_id_trian = np.delete(mouse_id_full, mouse_id_test-1)
    mouse_id_train = list(mouse_id_trian)
    
    # load training dataset
    idx = 0
    for mouse_id_load in mouse_id_train:
        idx = idx + 1
        fileName = "format4ML_GC6f_emx_0" + str(mouse_id_load) + "_windowLen" + str(window_length) + "_winStep_0" + str(window_step) + "_v2_threshold_10.csv"
        loadPath = "/Users/lizhu/Dropbox/projects/calcium/format4ML/" + fileName
#         current_progress = ("\n----------------------------------------\nloaded--> mouse_id: %s ..." \
#                                         % (mouse_id_load))
#         print current_progress
        comb = gl.SFrame.read_csv(loadPath, delimiter=',',header=False,verbose = False)
        colName_dg = 'degree'
        colName_dg = gl.SArray([colName_dg + repr(i+1) for i in range(30)])
        colName_cc = 'clusteringCoef'
        colName_cc = gl.SArray([colName_cc + repr(i+1) for i in range(30)])
        colName_pl = 'pathlength'
        colName_pl = gl.SArray([colName_pl + repr(i+1) for i in range(30)])
        colName = colName_dg.append(colName_cc.append(colName_pl.append(gl.SArray(['Whisker']))))
        colName = (list(colName))
        dictionary = dict(zip(comb.column_names(), colName))
        comb = comb.rename(dictionary)
        comb = gl.toolkits.cross_validation.shuffle(comb, random_seed=1)
        # comb_train['Whisker'].show(view = 'Categorical')
        
        if idx == 1:
            comb_train = comb
        else:
            comb_train = comb_train.append(comb)
                    
    # load testing dataset
    fileName = "format4ML_GC6f_emx_0" + str(mouse_id_test) + "_windowLen" + str(window_length) + "_winStep_0" + str(window_step) + "_v2_threshold_10.csv"
    loadPath = "/Users/lizhu/Dropbox/projects/calcium/format4ML/" + fileName
#     current_progress = ("\n----------------------------------------\nloaded--> mouse_id: %s ..." \
#                                         % (mouse_id_test))
#     print current_progress
    comb = gl.SFrame.read_csv(loadPath, delimiter=',',header=False,verbose = False)
    colName_dg = 'degree'
    colName_dg = gl.SArray([colName_dg + repr(i+1) for i in range(30)])
    colName_cc = 'clusteringCoef'
    colName_cc = gl.SArray([colName_cc + repr(i+1) for i in range(30)])
    colName_pl = 'pathlength'
    colName_pl = gl.SArray([colName_pl + repr(i+1) for i in range(30)])
    colName = colName_dg.append(colName_cc.append(colName_pl.append(gl.SArray(['Whisker']))))
    colName = (list(colName))
    dictionary = dict(zip(comb.column_names(), colName))
    comb = comb.rename(dictionary)
    comb_test = gl.toolkits.cross_validation.shuffle(comb, random_seed=1)
        
    
    return comb_train, comb_test

In [62]:
# train and cross-validation
def lz_logistic_across_subjects_AC_SE_SP(data_train, data_test, feature_type, l2_penalty, l1_penalty):
    
    # clearify features
    feature_dg = data_train.column_names()[0:30] # feature 1~30: degree, 31~60: clustering coefficient, 60~90: pathlength
    feature_cc = data_train.column_names()[30:60]
    feature_pl = data_train.column_names()[60:90]
    
    if feature_type == 1:   feature = feature_dg
    if feature_type == 2:   feature = feature_cc
    if feature_type == 3:   feature = feature_pl
    if feature_type == 12:  feature = feature_dg + feature_cc
    if feature_type == 13:  feature = feature_dg + feature_pl
    if feature_type == 23:  feature = feature_cc + feature_pl
    if feature_type == 123: feature = feature_dg + feature_cc + feature_pl

    m = gl.logistic_classifier.create(data_train,
                                      target='Whisker',
                                      features=feature,
                                      l2_penalty = l2_penalty, 
                                      l1_penalty = l1_penalty,
                                      validation_set=None,
                                      verbose = False)
    confusion_matrix = m.evaluate(data_test, 'confusion_matrix')
    confusion_matrix = confusion_matrix.values()[0]

    TP, TN, FP, FN = lz_extract_ACC_SE_SP(confusion_matrix)

    SP = float(TN) / (TN + FP)
    SE = float(TP) / (TP + FN)
    AC = float(TP+TN) / (TP+TN+FP+FN)
    
    print 'AC = ', AC
    print 'SE = ', SE
    print 'SP = ', SP
    
    return AC, SE, SP

In [60]:
def lz_extract_ACC_SE_SP(confusion_matrix):
    TP = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==1)]
    if np.size(TP) == 0:
        TP = 0
    else:
        TP = TP['count'][0]
    TN = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==0)]
    if np.size(TN) == 0:
        TN = 0
    else:
        TN = TN['count'][0]
    FP = confusion_matrix[(confusion_matrix['target_label']==0) & (confusion_matrix['predicted_label']==1)]
    if np.size(FP) == 0:
        FP = 0
    else:
        FP = FP['count'][0]
    FN = confusion_matrix[(confusion_matrix['target_label']==1) & (confusion_matrix['predicted_label']==0)]
    if np.size(FN) == 0:
        FN = 0
    else:
        FN = FN['count'][0]
    
    return TP, TN, FP, FN